# SALESFORCE_APIS

## Overview
`SALESFORCE_APIS` is provided as an example function to demonstrate how to retrieve available Salesforce REST APIs for a given instance and access token. You can modify this example to query any Salesforce APIs for more useful or specific data, adapting it to your integration or automation needs.  For more information on available endpoints and how to use the Salesforce REST API, see the [Salesforce REST API Quick Start](https://developer.salesforce.com/docs/atlas.en-us.api_rest.meta/api_rest/quickstart_oauth.htm).

## Usage
To use the `SALESFORCE_APIS` function in Excel, enter it as a formula in a cell, specifying your Salesforce instance URL and OAuth access token:

```excel
=SALESFORCE_APIS(instance_url, token)
```
- `instance_url`: Your Salesforce instance URL (e.g., https://MyDomainName.my.salesforce.com)
- `token`: Your Salesforce OAuth access token

## Arguments
| Argument      | Type   | Required/Optional | Description                                                        | Example |
|--------------|--------|-------------------|--------------------------------------------------------------------|---------|
| instance_url | string | Required          | Salesforce instance URL (e.g., https://MyDomainName.my.salesforce.com) | "https://MyDomainName.my.salesforce.com" |
| token        | string | Required          | Salesforce OAuth access token                                      | "00Dxx0000001gPFTAY!AR8AQ..." |

## Returns
| Return Value | Type    | Description                                      | Example |
|--------------|---------|--------------------------------------------------|---------|
| result       | 2D list | 2D list with columns [Service, URL Path]. First row is headers. | [["Service", "URL Path"], ["sobjects", "/services/data/v63.0/sobjects"], ...] |
| Error        | string  | Error message if credentials are invalid or missing | "Invalid Salesforce instance_url or token. (HTTP error)" |

## Example

**Sample Input:**
- `instance_url`: https://MyDomainName.my.salesforce.com
- `token`: 00Dxx0000001gPFTAY!AR8AQ... (your OAuth token)

```excel
=SALESFORCE_APIS("https://MyDomainName.my.salesforce.com", "00Dxx0000001gPFTAY!AR8AQ...")
```

**Sample Output:**
| Service         | URL Path                      |
|-----------------|------------------------------|
| sobjects        | /services/data/v63.0/sobjects |
| tooling         | /services/data/v63.0/tooling  |
| ...             | ...                           |

If credentials are invalid:
| You need to replace Salesforce instance_url and token with your own. |

## How to Acquire a Salesforce Access Token

See the [Salesforce REST API Quick Start](https://developer.salesforce.com/docs/atlas.en-us.api_rest.meta/api_rest/quickstart_oauth.htm)

To use the `SALESFORCE_APIS` function, you need a valid Salesforce OAuth access token. Follow these steps to obtain one using the Salesforce CLI:

### 1. Prerequisites
- Install the Salesforce CLI and ensure it is available in your PATH.
- Have a Salesforce user account with API access.

### 2. Authenticate to Your Org
Open Windows PowerShell and run:

```powershell
sf org login web --alias dev-instance
```
- This opens a browser window for login. After login, credentials are stored under the alias `dev-instance`.

### 3. Fetch the Access Token
Run the following command to display your access token:

```powershell
sf org display --target-org dev-instance --json
```
- Look for the `accessToken` field in the output. This is your API key for use in Excel or API calls.

### 4. Update Your Usage
- Copy the new access token and use it as the `token` parameter in the `SALESFORCE_APIS` function.

In [ ]:
import requests
from requests.exceptions import HTTPError

def salesforce_apis(instance_url, token):
    """
    Retrieves available Salesforce REST APIs and returns them as a 2D list.

    Args:
        instance_url (str): Salesforce instance URL (e.g., https://MyDomainName.my.salesforce.com)
        token (str): Salesforce OAuth access token

    Returns:
        list: 2D list with columns [Service, URL Path]. First row contains headers, or a string error message if credentials are invalid or missing.
    """
    # Ensure instance_url doesn't end with a slash
    instance_url = instance_url.rstrip('/')
    proxy_url = "https://cors.boardflare.com/"
    # Construct the endpoint URL
    url = f"{proxy_url}{instance_url}/services/data/v63.0/"
    # Set up the headers with the authorization token
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        data = response.json()
        result = [["Service", "URL Path"]]
        for service_name, url_path in data.items():
            result.append([service_name, url_path])
        return result
    except HTTPError:
        return "Invalid Salesforce instance_url or token. (HTTP error)"
    except Exception:
        return "An error occurred. Please check your instance_url and token."

In [ ]:
import ipytest
ipytest.autoconfig()

def test_demo_valid():
    # This test is for demonstration; replace with valid credentials to run live
    instance_url = "https://orgfarm-27d44b5d7b-dev-ed.develop.my.salesforce.com"
    token = "your-salesforce-token"
    result = salesforce_apis(instance_url, token)
    assert isinstance(result, list) or isinstance(result, str)

def test_invalid_token():
    instance_url = "https://MyDomainName.my.salesforce.com"
    token = "invalid_token"
    result = salesforce_apis(instance_url, token)
    assert isinstance(result, str)
    assert "invalid" in result.lower() or "error" in result.lower()

def test_invalid_url():
    instance_url = "https://notareal.salesforce.com"
    token = "test_token"
    result = salesforce_apis(instance_url, token)
    assert isinstance(result, str)
    assert "invalid" in result.lower() or "error" in result.lower()

def test_missing_token():
    instance_url = "https://MyDomainName.my.salesforce.com"
    token = ""
    result = salesforce_apis(instance_url, token)
    assert isinstance(result, str)
    assert "invalid" in result.lower() or "error" in result.lower()

ipytest.run()

In [ ]:
# Interactive Demo
import gradio as gr

demo_cases = [
    [
        "https://MyDomainName.my.salesforce.com",
        "00Dxx0000001gPFTAY!AR8AQ..."
    ]
]

demo = gr.Interface(
    fn=salesforce_apis,
    inputs=[
        gr.Textbox(label="Salesforce Instance URL", value="https://MyDomainName.my.salesforce.com"),
        gr.Textbox(label="OAuth Access Token", value="00Dxx0000001gPFTAY!AR8AQ...")
    ],
    outputs=gr.Dataframe(headers=["Service", "URL Path"], label="Available APIs"),
    examples=demo_cases,
    description="Retrieve available Salesforce REST APIs for a given instance and access token.",
    flagging_mode="never",
)
demo.launch()